# Using the Edge Impulse Python SDK to Upload and Download Data

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://docs.edgeimpulse.com/docs/edge-impulse-python-sdk/python-sdk-upload-download"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-ei-32px.png" /> View on edgeimpulse.com</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-upload-download.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-colab-32px.png" /> Run in Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-upload-download.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-github-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/notebooks/python-sdk-upload-download.ipynb" download><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/icon-download-32px.png" /> Download notebook</a>
  </td>
</table>

TODO: write description

> **WARNING:** This notebook will delete data in your Edge Impulse project, so be careful! We recommend creating a throwaway project when testing this notebook.

In [ ]:
# If you have not done so already, install the following dependencies
# !python -m pip install edgeimpulse

In [ ]:
import io

import edgeimpulse as ei

You will need to obtain an API key from an Edge Impulse project. Log into [edgeimpulse.com](https://edgeimpulse.com/) and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**.

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-copy-ei-api-key.png)

Note that you do not actually need to use the project in the Edge Impulse Studio. We just need the API Key.

Paste that API key string in the `ei.API_KEY` value in the following cell:

In [ ]:
# Settings
ei.API_KEY = "ei_47649fc7d8b94f9fbf64b9abcf4013845edb8fbfb556bb151663165c36ee4555" # Change this to your Edge Impulse API key

## Upload JSON data

One way to upload data is to encode it in JSON format. See the [data acquisition format specificaion](https://docs.edgeimpulse.com/reference/data-acquisition-format#data-acquisition-format-specification) for more information on acceptable key/value pairs. Note that at this time, the `signature` value can be set to `0`. The raw data must be encoded in an IO object.

In [ ]:
import json

In [ ]:
# Create two different example data samples
sample_data_1 = {
    "protected": {
        "ver": "v1",
        "alg": "none",
    },
    "signature": 0,
    "payload": {
        "device_name": "ac:87:a3:0a:2d:1b",
        "device_type": "DISCO-L475VG-IOT01A",
        "interval_ms": 10,
        "sensors": [
            { "name": "accX", "units": "m/s2" },
            { "name": "accY", "units": "m/s2" },
            { "name": "accZ", "units": "m/s2" }
        ],
        "values": [
            [ -9.81, 0.03, 0.21 ],
            [ -9.83, 0.04, 0.27 ],
            [ -9.12, 0.03, 0.23 ],
            [ -9.14, 0.01, 0.25 ]
        ]
    }
}
sample_data_2 = {
    "protected": {
        "ver": "v1",
        "alg": "none",
    },
    "signature": 0,
    "payload": {
        "device_name": "ac:87:a3:0a:2d:1b",
        "device_type": "DISCO-L475VG-IOT01A",
        "interval_ms": 10,
        "sensors": [
            { "name": "accX", "units": "m/s2" },
            { "name": "accY", "units": "m/s2" },
            { "name": "accZ", "units": "m/s2" }
        ],
        "values": [
            [ -9.56, 5.34, 1.21 ],
            [ -9.43, 1.37, 1.27 ],
            [ -9.22, -4.03, 1.23 ],
            [ -9.50, -0.98, 1.25 ]
        ]
    }
}

In [ ]:
# Provide a filename, category, label, and optional metadata for each sample
my_samples = [
    {
        "filename": "001.json",
        "data": io.BytesIO(json.dumps(sample_data_1).encode('utf-8')),
        "category": "training",
        "label": "idle",
        "metadata": {
            "source": "accelerometer",
            "collection site": "desk",
        },
    },
    {
        "filename": "002.json",
        "data": io.BytesIO(json.dumps(sample_data_2).encode('utf-8')),
        "category": "training",
        "label": "wave",
        "metadata": {
            "source": "accelerometer",
            "collection site": "desk",
        },
    },
]

In [ ]:
# Wrap the samples in instances of the Sample class
samples = [ei.data.sample_type.Sample(**i) for i in my_samples]

# Upload samples to your project
successes, fails = ei.data.upload_samples(samples)

# TODO: Check successes and fails

If you head to the *Data acquisition* page on your project, you should see your 

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-upload-download-json-data.png)